# Imports

In [1]:
import os
import packages

from context.utils import typer as t

from toolkit.utils import utils
from toolkit.utils.utils import rp_print
from toolkit.utils.llm import main as utils_llm

import context.instances as inst
import context.consts as const
import context.settings.main as settings_main

from toolkit.llm.langchain.core import integration, utils as utils_lc
from toolkit.llm.langchain.data.indexing import (
  documents, document_loaders, text_splitters,
)
from toolkit.llm.langchain.execution import runnables, graphs, tools, agents
from toolkit.llm.langchain.models import prompts, llms, messages

2025-02-12 07:51:12.918 | INFO     | packages:<module>:68 - apps directory: /home/hieudt71/IvyEdge/apps
2025-02-12 07:51:12.919 | INFO     | packages:<module>:69 - Toolkit path: /home/hieudt71/IvyEdge/apps/toolkit
2025-02-12 07:51:12.920 | INFO     | packages:<module>:70 - Environment files loaded:
2025-02-12 07:51:12.921 | INFO     | packages:<module>:72 -   - /home/hieudt71/IvyEdge/apps/ports.env
2025-02-12 07:51:12.922 | INFO     | packages:<module>:72 -   - /home/hieudt71/IvyEdge/apps/.env


/home/nvidia/miniconda3/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-12 07:51:17.183 | INFO     | toolkit.db.mongodb:connect:436 - Successfully connected to MongoDB database: app
2025-02-12 07:51:17.185 | INFO     | toolkit.db.mongodb:get_collection:447 - Creating manager for collection: vehicle
2025-02-12 07:51:17.187 | INFO     | toolkit.db.redix:connect:415 - Successfully connected to Redis database: 0
2025-02-12 07:51:22.052 | WARNING  | context.settings.main:<module>:18 - Running in DEVELOPMENT mode
2025-02-12 07:51:22.472 | INFO     | toolkit.llm.langchain.models.llms:create_llm:78 - 🔹 LLM OpenAI gpt-4o-mini
2025-02-12 07:51:22.678 | INFO     | toolkit.llm.langchain.models.llms:create_llm:78 - 🔹 LLM Google gemini-2.0-flash-exp
2025-02-12 07:51:22.900 | INFO     | toolkit.llm.la

# Data

## Vehicle Owner's Manual

In [3]:
os.getenv("LD_PRELOAD")

'/lib/aarch64-linux-gnu/libGLdispatch.so.0'

In [4]:
prompt_extract_headers = """\
Analyze the following document and extract a list of its section headers or titles, inferring them from the content. Return the headers/titles as a Python list (e.g., ['header 1', 'header 2', 'header 3']) and nothing more.

Input: INPUT

Output:
"""

prompt_assign_headers_to_snippet = """\
Given a snippet of text from a larger document and a list of existing headers from that document, analyze the text (no code) and determine which of the provided headers (or which combination of headers) best applies to the content of the snippet. Return the appropriate header(s) as a Python list (e.g., ['Header 1'] or ['Header 2', 'Header 3']) . If none of the provided headers is a good fit, return an empty list ([]).

Input Snippet: SNIPPET

Input Headers: HEADERS

Output:
"""

loader = document_loaders.PyPDFLoader(
	file_path=f"{packages.APP_PATH}/data/projects/manual.pdf"
)
docs = loader.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	chunk_size=1000, chunk_overlap=200, add_start_index=True,
)
splits = text_splitter.split_documents(docs)

In [ ]:
# # Optional
# headers = inst.llm_main.invoke(prompt_extract_headers.replace("INPUT", docs[0].page_content))

# for split in utils.tqdm(splits):
# 	split_headers = inst.llm_main.invoke(prompt_assign_headers_to_snippet\
# 											.replace("SNIPPET", split.page_content)\
# 											.replace("HEADERS", headers.content))
# 	split.metadata["headers"] = split_headers.content

In [6]:
document_ids = inst.vector_stores_qdrant["vehicle"].add_documents(documents=splits)

## User Query Category

### Pre-Process

In [7]:
base_path = f"{packages.APP_PATH}/data/projects/vehicle"

# Process car control data
data_control = utils.read_json_file(f"{base_path}/questions/control.json")["questions"]
control_result = []
for category in data_control:
    for question in category["questions"]:
        control_result.append({
            "input": question,
            "output": "car_control"
        })
utils.save_to_json(control_result, f"{base_path}/examples/user_query_category/control.json")

# Process car manual data
data_manual = utils.read_json_file(f"{base_path}/questions/manual.json")
manual_result = [
    {"input": item["question"], "output": "car_manual"}
    for item in data_manual
    if "question" in item
]
utils.save_to_json(manual_result, f"{base_path}/examples/user_query_category/manual.json")

Data successfully saved to /home/hieudt71/IvyEdge/apps/data/projects/vehicle/examples/user_query_category/control.json
Data successfully saved to /home/hieudt71/IvyEdge/apps/data/projects/vehicle/examples/user_query_category/manual.json


### VectorDB

In [8]:
base_path = f"{packages.APP_PATH}/data/projects/vehicle/examples/user_query_category"
file_paths = os.listdir(base_path)
file_paths = [f"{base_path}/{file_path}" for file_path in file_paths]

for file_path in utils.tqdm(file_paths):
		data_lst = utils.read_json_file(file_path)
		data_lst = [str(item) for item in data_lst]
		docs = [documents.Document(page_content=t) for t in data_lst]
		
		splits = docs

		document_ids = inst.vector_stores_qdrant["user_query_category"].add_documents(documents=splits)

100%|██████████| 5/5 [00:22<00:00,  4.58s/it]


In [ ]:
# Test
retriever = inst.vector_stores_qdrant["user_query_category"].as_retriever()

tests = [
	"Activate the AC mode. Increase front wiper speed. Help me Unlock doors",
	"Yes",
]

user_query = tests[1]

result = retriever.invoke(user_query)

rp_print(result)

## Vehicle Properties Field Paths

In [9]:
from toolkit.db.mongodb import MongoDBCollectionManager, MongoDBDatabaseManager
import json
import asyncio

"""Test inserting vehicle data into MongoDB."""
print("\n=== Testing Vehicle Data Insert ===")

# Initialize database connection
db = MongoDBDatabaseManager(
		username="root",
		password="example",
		host="localhost",
		port="27017",
		db_name="app"
)

# Get the vehicles collection
vehicles_collection = db.get_collection("vehicles")

# Clean up: drop the collection first
print("\nCleaning up: dropping existing collection...")
drop_result = await vehicles_collection.drop_collection()
print(f"Drop collection result: {drop_result}")

# Insert the JSON file
print("\nInserting vehicle data...")
path = f"{packages.ROOT_PATH}/apps/data/projects/vehicle/vehicle_data.json"
result = await vehicles_collection.insert_json_file(path, doc_id="v123")
print(f"Insert result: {result}")

if result["success"]:
		# Verify the insertion
		print("\nVerifying insertion...")
		doc = await vehicles_collection.get_full_document(result["inserted_id"])
		print(f"\nVerification - Retrieved document:")
		print(json.dumps(doc, indent=2, default=MongoDBCollectionManager.json_encode))

field_paths = await vehicles_collection.get_all_field_paths("v123")
field_paths = field_paths["field_paths"]
data_lst = field_paths

data_lst = [str(item) for item in data_lst]
docs = [documents.Document(page_content=t) for t in data_lst]

splits = docs

document_ids = inst.vector_stores_qdrant["vehicle_properties_field_paths"].add_documents(documents=splits)

await db.close()

2025-02-12 07:54:55.830 | INFO     | toolkit.db.mongodb:connect:436 - Successfully connected to MongoDB database: app
2025-02-12 07:54:55.832 | INFO     | toolkit.db.mongodb:get_collection:447 - Creating manager for collection: vehicles
2025-02-12 07:54:55.834 | WARNING  | toolkit.db.mongodb:drop_collection:19 - Dropping collection: vehicles
2025-02-12 07:54:55.856 | INFO     | toolkit.db.mongodb:drop_collection:21 - Successfully dropped collection: vehicles
2025-02-12 07:54:55.858 | INFO     | toolkit.db.mongodb:insert_json_file:362 - Reading JSON file from: /home/hieudt71/IvyEdge/apps/data/projects/vehicle/vehicle_data.json
2025-02-12 07:54:55.879 | INFO     | toolkit.db.mongodb:insert_json_file:377 - Successfully inserted document with ID: v123
2025-02-12 07:54:55.880 | INFO     | toolkit.db.mongodb:get_full_document:245 - Retrieving full document for v123
2025-02-12 07:54:55.884 | INFO     | toolkit.db.mongodb:get_full_document:249 - Successfully retrieved full document for v123
20


=== Testing Vehicle Data Insert ===

Cleaning up: dropping existing collection...
Drop collection result: {'success': True, 'message': 'Dropped collection: vehicles'}

Inserting vehicle data...
Insert result: {'success': True, 'inserted_id': 'v123', 'message': 'Successfully inserted document from /home/hieudt71/IvyEdge/apps/data/projects/vehicle/vehicle_data.json'}

Verifying insertion...

Verification - Retrieved document:
{
  "success": true,
  "document": {
    "_id": "v123",
    "vin": "1HGCM82633A123456",
    "metadata": {
      "make": "Toyota",
      "model": "Camry",
      "year": 2024,
      "type": "sedan",
      "color": "silver",
      "features": [
        "hybrid",
        "automatic",
        "sunroof",
        "lane_assist",
        "adaptive_cruise"
      ],
      "registration": {
        "plate": "ABC123",
        "state": "CA",
        "expirationDate": "2024-12-31T00:00:00Z"
      }
    },
    "currentState": {
      "lastUpdated": "2024-02-05T10:30:00Z",
      "i

2025-02-12 07:55:12.340 | INFO     | toolkit.db.mongodb:close:455 - MongoDB connection closed
